In [20]:
import numpy as np
from PIL import Image
import filepath

def create_checkerboard_(size, num_squares):
    # size 是图像的尺寸 (width, height)
    # num_squares 是每行和每列的方格数量

    # 计算每个方格的尺寸
    square_size = size[0] // num_squares
    
    # 创建一个空的 numpy 数组
    checkerboard = np.zeros((size[1], size[0]), dtype=np.uint8)
    
    # 填充 checkerboard 数组
    for y in range(num_squares):
        for x in range(num_squares):
            if (x + y) % 2 == 0:
                checkerboard[y*square_size:(y+1)*square_size, x*square_size:(x+1)*square_size] = 255
    
    # 将 numpy 数组转换为 PIL 图像
    image = Image.fromarray(checkerboard, 'L')
    
    return image

def create_checkerboard(size, square_size:int):
    size = (int(size[0]),int(size[1]))
    square_size = int(square_size)
    # size 是图像的尺寸 (width, height)
    # num_squares 是每行和每列的方格数量

    # 计算每个方格的尺寸
    # square_size = size[0] // num_squares
    num_squares_x = int(np.ceil(size[0] / square_size))
    num_squares_y = int(np.ceil(size[1] / square_size))
    
    # 创建一个空的 numpy 数组
    checkerboard = np.zeros((size[1], size[0]), dtype=np.uint8)
    
    # 填充 checkerboard 数组
    for y in range(num_squares_y):
        for x in range(num_squares_x):
            if (x + y) % 2 == 0:
                checkerboard[y*square_size:(y+1)*square_size, x*square_size:(x+1)*square_size] = 255
    
    # 将 numpy 数组转换为 PIL 图像
    image = Image.fromarray(checkerboard, 'L')
    
    return image

# 使用示例


In [21]:
np.ceil(.1)

1.0

In [22]:
square_size = 512/8
size = (512, 512+square_size)  # 图像尺寸 (宽度, 高度)
# num_squares = 8    # 每行和每列的方格数量


# 创建棋盘格图像
# checkerboard_image = create_checkerboard_(size, num_squares)
checkerboard_image = create_checkerboard(size, square_size)
# 保存图像
checkerboard_image.save('checkerboard.png')

# 显示图像（可选）
checkerboard_image.show()


(eog:151858): Gtk-WARNING **: 08:38:29.361: Theme file for DMZ-White has no directories


In [23]:
help(Image.fromarray)

Help on function fromarray in module PIL.Image:

fromarray(obj, mode=None)
    Creates an image memory from an object exporting the array interface
    (using the buffer protocol)::
    
      from PIL import Image
      import numpy as np
      a = np.zeros((5, 5))
      im = Image.fromarray(a)
    
    If ``obj`` is not contiguous, then the ``tobytes`` method is called
    and :py:func:`~PIL.Image.frombuffer` is used.
    
    In the case of NumPy, be aware that Pillow modes do not always correspond
    to NumPy dtypes. Pillow modes only offer 1-bit pixels, 8-bit pixels,
    32-bit signed integer pixels, and 32-bit floating point pixels.
    
    Pillow images can also be converted to arrays::
    
      from PIL import Image
      import numpy as np
      im = Image.open("hopper.jpg")
      a = np.asarray(im)
    
    When converting Pillow images to arrays however, only pixel values are
    transferred. This means that P and PA mode images will lose their palette.
    
    :param o

In [18]:
type(checkerboard_image)

PIL.Image.Image

In [24]:
from util.texture import *